In [1]:
#cell block 1
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from prod_feature_extraction import *
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#cell block 2
#define functions to test vehicle detections in an image
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True, hog_color_space="HSV"):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_color_space != 'RGB':
            if hog_color_space == 'HSV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            elif hog_color_space == 'LUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
            elif hog_color_space == 'HLS':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
            elif hog_color_space == 'YUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
            elif hog_color_space == 'YCrCb':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(img)   
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        elif hog_channel=='GRAY' :
                gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                hog_features = get_hog_features(gray, orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features

    return np.concatenate(img_features)
def search_windows(img, windows, clf, scaler, model, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    i=1
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
        
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        
        #test_features = scaler.transform(features)
        #6) Predict using your classifier
        prediction = clf.predict_classes(model.transform(test_features),verbose=0)
        #7) If positive (prediction == 1) then save the window
        if prediction[0] == 1:
            on_windows.append(window)
        
            
            
    #8) Return windows for positive detections
    return on_windows
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,255,0), 6)
    # Return the image
    return img
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 255, 0), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [ ]:
#cell block 3
#test LinearSVC classifier for vehcile detection
import pickle

#load classifier
with open('linearsvc_vehicle_classifier.pkl', 'rb') as f:
    classifier_dict = pickle.load(f)
test_images = glob.glob('test_images/*.jpg')
clf=classifier_dict["classifier"]
model=classifier_dict["feature_select"]
X_scaler = classifier_dict["scaler"]
#feature extraction parameters
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
hog_color_space="HSV"
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (24,24) # Spatial binning dimensions
hist_bins = 20    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

i=1
for img_path in test_images:
    image = mpimg.imread(img_path)
    draw_image = np.copy(image)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    org_image=np.copy(image)
    image = image.astype(np.float32)/255

    windows = slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.85, 0.8))
    
   
    
    windows.extend(slide_window(image, x_start_stop=[500, 1000], y_start_stop=[380,500], 
                        xy_window=(60, 60), xy_overlap=(0.65, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(80, 80), xy_overlap=(0.65, 0.6)))
                        
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(90, 90), xy_overlap=(0.60, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(110, 110), xy_overlap=(0.50, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-230, None], y_start_stop=[380,600], 
                        xy_window=(110, 110), xy_overlap=(0.80, 0.7)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(90, 90), xy_overlap=(0.80, 0.7)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-150, None], y_start_stop=[380,700], 
                        xy_window=(150, 150), xy_overlap=(0.71, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[400, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.7, 0.8)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-480, None], y_start_stop=[380,700], 
                        xy_window=(160, 160), xy_overlap=(0.9, 0.8)))
    """                    
    windows = slide_window(image, x_start_stop=[300, None], y_start_stop=[380,700], 
                        xy_window=(160, 160), xy_overlap=(0.5, 0.5))
                        
    windows = slide_window(image, x_start_stop=[400, None], y_start_stop=[380,600], 
                        xy_window=(150, 150), xy_overlap=(0.7, 0.5))
                        
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[370,600], 
                        xy_window=(80, 80), xy_overlap=(0.6, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[400, None], y_start_stop=[370,600], 
                        xy_window=(80, 80), xy_overlap=(0.5, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[400, None], y_start_stop=[370,600], 
                        xy_window=(90, 90), xy_overlap=(0.5, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[800, None], y_start_stop=[450,600], 
                        xy_window=(80, 80), xy_overlap=(0.7, 0.7)))
    
    windows.extend(slide_window(image, x_start_stop=[400, None], y_start_stop=[370,600], 
                        xy_window=(100, 100), xy_overlap=(0.8, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[700, None], y_start_stop=[370,600], 
                        xy_window=(120, 120), xy_overlap=(0.9, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[200, None], y_start_stop=[350,700], 
                        xy_window=(135, 135), xy_overlap=(0.9, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[200, None], y_start_stop=[350,700], 
                        xy_window=(150, 150), xy_overlap=(0.9, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[200, None], y_start_stop=[350,700], 
                        xy_window=(200, 200), xy_overlap=(0.9, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[200, None], y_start_stop=[350,700], 
                        xy_window=(250, 250), xy_overlap=(0.9, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[200, None], y_start_stop=[400,700], 
                        xy_window=(300, 300), xy_overlap=(0.9, 0.6)))"""
    
    

    
    
    hot_windows = search_windows(image, windows, clf, X_scaler, model, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    print(hot_windows)
    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

    plt.imshow(window_img)
    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,2)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    print(heatmap[heatmap>0.0])
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(org_image, labels)
    plt.imshow(draw_img)
    plt.title('Car Positions')
    
    plt.show()

    
    """plt.imshow(draw_img)
    plt.title('Car Positions')
    
    plt.savefig("testimg"+str(i)+".jpg")
    plt.imshow(window_img)
    plt.title('Detected Windows Positions')
    
    plt.savefig("testimg_window"+str(i)+".jpg")
    i+=1"""

In [3]:
#cell block 4
#keras vehicle detection
import pickle


with open('linearsvc_vehicle_classifier.pkl', 'rb') as f:
    classifier_dict = pickle.load(f)
test_images = glob.glob('test_images/*.jpg')
model=classifier_dict["feature_select"]

clf=load_model("model_alt.h5")
X_scaler = classifier_dict["scaler"]
### TODO: Tweak these parameters and see how the results change.
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (24, 24) # Spatial binning dimensions
hist_bins = 20    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
i=1
for img_path in test_images:
    image = mpimg.imread(img_path)
    draw_image = np.copy(image)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    org_image=np.copy(image)
    image = image.astype(np.float32)/255

    windows = slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.85, 0.8))
    
   
    
    #windows.extend(slide_window(image, x_start_stop=[500, 1000], y_start_stop=[380,500], 
    #                    xy_window=(60, 60), xy_overlap=(0.65, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(80, 80), xy_overlap=(0.7, 0.7)))
                        
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(90, 90), xy_overlap=(0.60, 0.7)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(110, 110), xy_overlap=(0.50, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-230, None], y_start_stop=[380,600], 
                        xy_window=(110, 110), xy_overlap=(0.80, 0.75)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(90, 90), xy_overlap=(0.80, 0.75)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-150, None], y_start_stop=[380,700], 
                        xy_window=(150, 150), xy_overlap=(0.71, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[400, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.8, 0.8)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-480, None], y_start_stop=[380,700], 
                        xy_window=(160, 160), xy_overlap=(0.8, 0.8)))
    

    
    
    hot_windows = search_windows(image, windows, clf, X_scaler,model, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    
    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

    
    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)
    plt.imshow(heat,cmap='hot' )
    plt.title('Heatmap of Hot Windows')
    
    plt.savefig("output_images/heat_window"+str(i)+".jpg")
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(org_image, labels)


    
    #plt.imshow(draw_img)
    #plt.title('Car Positions')
    
    #plt.show()
    """
    plt.imshow(draw_img)
    plt.title('Car Positions')
    
    plt.savefig("testimg"+str(i)+".jpg")
    plt.imshow(window_img)
    plt.title('Detected Windows Positions')
    
    plt.savefig("testimg_window"+str(i)+".jpg")
    
    plt.imshow(draw_boxes(draw_image, windows, color=(0, 0, 255), thick=6) )
    plt.title('Search Windows')
    
    plt.savefig("search_window"+str(i)+".jpg")
    
    
    plt.imshow(labels[0], cmap='gray')
    plt.title('Filtered and Grouped Hot Windows')
    
    plt.savefig("fitlered_heat_window"+str(i)+".jpg")
   
    """
    i+=1

In [ ]:
#cell block 5
#testing video pipleine for keras
import pickle

with open('linearsvc_vehicle_classifier.pkl', 'rb') as f:
    classifier_dict = pickle.load(f)


clf=load_model("model_alt.h5")
X_scaler = classifier_dict["scaler"]
### TODO: Tweak these parameters and see how the results change.
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (24, 24) # Spatial binning dimensions
hist_bins = 20    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [500, None] # Min and max in y to search in slide_window()
def video_process_lines(img):
    image = img
    draw_image = np.copy(image)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    org_image=np.copy(image)
    image = image.astype(np.float32)/255

    windows = slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.85, 0.8))
    
   
    
    #windows.extend(slide_window(image, x_start_stop=[500, 1000], y_start_stop=[380,500], 
    #                    xy_window=(60, 60), xy_overlap=(0.65, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(80, 80), xy_overlap=(0.7, 0.7)))
                        
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(90, 90), xy_overlap=(0.60, 0.7)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(110, 110), xy_overlap=(0.50, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-230, None], y_start_stop=[380,600], 
                        xy_window=(110, 110), xy_overlap=(0.80, 0.75)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(90, 90), xy_overlap=(0.80, 0.75)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-150, None], y_start_stop=[380,700], 
                        xy_window=(150, 150), xy_overlap=(0.71, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[400, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.8, 0.8)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-480, None], y_start_stop=[380,700], 
                        xy_window=(160, 160), xy_overlap=(0.8, 0.8)))
    

    
    
    hot_windows = search_windows(image, windows, clf, X_scaler,model, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    
    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

    
    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(org_image, labels)


    
    return draw_img
    

In [ ]:
#cell block 6
#run LinearSVC Pipeline
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output = 'deep_processed.mp4'
clip1 = VideoFileClip("project_video.mp4")
#clip1=clip1.subclip(12,17)
white_clip = clip1.fl_image(video_process_lines) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
#cell block 7
#test linearSVC video pipeline
with open('linearsvc_vehicle_classifier.pkl', 'rb') as f:
    classifier_dict = pickle.load(f)
test_images = glob.glob('test_images/*.jpg')
clf=classifier_dict["classifier"]
model=classifier_dict["feature_select"]
X_scaler = classifier_dict["scaler"]
### TODO: Tweak these parameters and see how the results change.
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
hog_color_space="HSV"
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (24,24) # Spatial binning dimensions
hist_bins = 20    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [500, None] # Min and max in y to search in slide_window()
def video_process_lines(img):
    image = img
    draw_image = np.copy(image)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    # Uncomment the following line if you extracted training
    # data from .png images (scaled 0 to 1 by mpimg) and the
    # image you are searching is a .jpg (scaled 0 to 255)
    org_image=np.copy(image)
    image = image.astype(np.float32)/255

    windows = slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.9, 0.8))
    
   
    
    windows.extend(slide_window(image, x_start_stop=[500, 1000], y_start_stop=[380,500], 
                        xy_window=(60, 60), xy_overlap=(0.65, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(80, 80), xy_overlap=(0.65, 0.6)))
                        
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(90, 90), xy_overlap=(0.60, 0.6)))
    
    windows.extend(slide_window(image, x_start_stop=[500, None], y_start_stop=[380,520], 
                        xy_window=(110, 110), xy_overlap=(0.50, 0.5)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-380, None], y_start_stop=[380,600], 
                        xy_window=(110, 110), xy_overlap=(0.75, 0.75)))
    
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-360, None], y_start_stop=[380,600], 
                        xy_window=(90, 90), xy_overlap=(0.65, 0.8)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-150, None], y_start_stop=[380,700], 
                        xy_window=(150, 150), xy_overlap=(0.9, 0.8)))
    
    windows.extend(slide_window(image, x_start_stop=[400, None], y_start_stop=[380,600], 
                        xy_window=(180, 180), xy_overlap=(0.7, 0.8)))
                        
    windows.extend(slide_window(image, x_start_stop=[image.shape[1]-480, None], y_start_stop=[380,700], 
                        xy_window=(160, 160), xy_overlap=(0.9, 0.8)))
    
    
    hot_windows = search_windows(image, windows, clf, X_scaler, model, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    

    
    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,3)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(org_image, labels)


    
    return draw_img

In [ ]:
#cell block 8
#run keras video pipleine
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output = 'project_processed.mp4'
clip1 = VideoFileClip("project_video.mp4")
#clip1=clip1.subclip(15,17)
white_clip = clip1.fl_image(video_process_lines) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)